# Levenstein Distance
## Paweł Kruczkiewicz
#### 21.04.2021 r.

Zadanie dotyczy wykorzystania odległości edycyjnej.

1. Zaimplementuj algorytm obliczania odległości edycyjnej w taki sposób, aby możliwe było określenie przynajmniej jednej sekwencji edycji (dodanie, usunięcie, zmiana znaku), która pozwala w minimalnej liczbie kroków, przekształcić jeden łańcuch w drugi.
2. Na podstawie poprzedniego punktu zaimplementuj prostą wizualizację działania algorytmu, poprzez wskazanie kolejnych wersji pierwszego łańcucha, w których dokonywana jest określona zmiana. "Wizualizacja" może działać w trybie tekstowym. Np. zmiana łańcuch "los" w "kloc" może być zrealizowana następująco:
        *k*los (dodanie litery k)
        klo*c* (zamiana s->c)
3. Przedstaw wynik działania algorytmu z p. 2 dla następujących par łańcuchów:
        los - kloc
        Łódź - Lodz
        kwintesencja - quintessence
        ATGAATCTTACCGCCTCG - ATGAGGCTCTGGCCCCTG
       
4. Zaimplementuj algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów.
5. Korzystając z gotowego tokenizera (np. spaCy - https://spacy.io/api/tokenizer) dokonaj podziału załączonego tekstu na tokeny.
6. Stwórz 2 wersje załączonego tekstu, w których usunięto 3% losowych tokenów.
7. Oblicz długość najdłuższego podciągu wspólnych tokenów dla tych tekstów.
8. Korzystając z algorytmu z punktu 4 skonstruuj narzędzie, o działaniu podobnym do narzędzia diff, tzn. wskazującego w dwóch plikach linie, które się różnią. Na wyjściu narzędzia powinny znaleźć się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać skąd dana linia pochodzi (< > - pierwszy/drugi plik) oraz numer linii w danym pliku.
9. Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

## Zad 1

In [1]:
def levenshteinDistance(text_a, text_b):
    delta = lambda x, y: 0 if x == y else 1

    len_a = len(text_a)
    len_b = len(text_b)

    dist_table = [[0 for _ in range(len_b + 1)] for _ in range(len_a + 1)]
    traceback_table = [[None for _ in range(len_b + 1)] for _ in range(len_a + 1)]

    for i in range(1, len_a + 1):
        dist_table[i][0] = i
        traceback_table[i][0] = "up"
    for j in range(1, len_b + 1):
        dist_table[0][j] = j
        traceback_table[0][j] = "left"

    for i in range(1, len_a + 1):
        for j in range(1, len_b + 1):
            x, y = text_a[i - 1], text_b[j - 1]  # current letters

            up_cost, left_cost, diag_cost = dist_table[i - 1][j] + 1, dist_table[i][j - 1] + 1, \
                                            dist_table[i - 1][j - 1] + delta(x, y)

            if up_cost < left_cost and up_cost < diag_cost:
                dist_table[i][j] = up_cost
                traceback_table[i][j] = "up"
            elif left_cost < diag_cost:
                dist_table[i][j] = left_cost
                traceback_table[i][j] = "left"
            else:
                dist_table[i][j] = diag_cost
                traceback_table[i][j] = "diag"

    return dist_table[len_a][len_b], traceback_table

## Zad 2

In [2]:
def editionVisualization(text_a, text_b):
    _, traceback_table = levenshteinDistance(text_a, text_b)
    i = len(text_a)
    j = len(text_b)

    steps = []
    curr_step = traceback_table[i][j]

    while curr_step is not None:
        steps.append(curr_step)

        if curr_step == "up":
            i -= 1
        elif curr_step == "left":
            j -= 1
        elif curr_step == "diag":
            i -= 1
            j -= 1
        else:
            raise AssertionError("Unknown Step")

        curr_step = traceback_table[i][j]

    steps.reverse()

    modified_string = text_a

    for step in steps:
        letter_a, letter_b = modified_string[i], text_b[j]
        if step == "up":
            print(f"{modified_string[:i]}\\{letter_a}/{modified_string[i+1:]}\t'{letter_a}' deleted")
            modified_string = modified_string[:i] + modified_string[i+1:]
        elif step == "left":
            print(f"{modified_string[:i]}+{letter_b}+{modified_string[i:]}\t'{letter_b}' added")
            modified_string = modified_string[:i] + letter_b + modified_string[i:]
            j += 1
            i += 1
        elif step == "diag":
            if letter_a != letter_b:
                print(f"{modified_string[:i]}*{letter_b}*{modified_string[i+1:]}\t'{letter_a}' -> '{letter_b}'")
                modified_string = modified_string[:i] + letter_b + modified_string[i+1:]
            i += 1
            j += 1

## Zad 3

In [3]:
editionVisualization("los", "kloc")
print()

editionVisualization( "Łódź", "Lodz")
print()

editionVisualization( "kwintesencja", "quintessence")
print()

editionVisualization("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")

+k+los	'k' added
klo*c*	's' -> 'c'

*L*ódź	'Ł' -> 'L'
L*o*dź	'ó' -> 'o'
Lod*z*	'ź' -> 'z'

*q*wintesencja	'k' -> 'q'
q*u*intesencja	'w' -> 'u'
quinte+s+sencja	's' added
quintessenc\j/a	'j' deleted
quintessenc*e*	'a' -> 'e'

ATGA*G*TCTTACCGCCTCG	'A' -> 'G'
ATGAG*G*CTTACCGCCTCG	'T' -> 'G'
ATGAGGCT+C+TACCGCCTCG	'C' added
ATGAGGCTCT*G*CCGCCTCG	'A' -> 'G'
ATGAGGCTCTG*G*CGCCTCG	'C' -> 'G'
ATGAGGCTCTGGC*C*CCTCG	'G' -> 'C'
ATGAGGCTCTGGCCCCT\C/G	'C' deleted


## Zad 4

In [4]:
def lcsequence(seq_a, seq_b):
    len_a = len(seq_a)
    len_b = len(seq_b)

    lcs = [[0 for _ in range(len_b + 1)] for _ in range(len_a + 1)]
    traceback = [[None for _ in range(len_b + 1)] for _ in range(len_a + 1)]

    for i in range(1, len_a + 1):
        lcs[i][0] = 0
        traceback[i][0] = "up"
    for j in range(1, len_b + 1):
        lcs[0][j] = 0
        traceback[0][j] = "left"

    for i in range(1, len_a + 1):
        for j in range(1, len_b + 1):
            x, y = seq_a[i - 1], seq_b[j - 1]  # current elements in sequences

            if x == y:
                lcs[i][j] = lcs[i - 1][j - 1] + 1
                traceback[i][j] = "diag"
            elif lcs[i - 1][j] >= lcs[i][j - 1]:
                lcs[i][j] = lcs[i - 1][j]
                traceback[i][j] = "up"
            else:
                lcs[i][j] = lcs[i][j - 1]
                traceback[i][j] = "left"

    return lcs[len_a][len_b], traceback



In [5]:
longest, traceback = lcsequence(["A", "B", "C", "B", "D", "A", "B"], ["B", "D", "C", "A", "B", "A"])
print(longest)

4


## Zad 5

In [6]:
# instalowanie pakietu w obecnym wirtualnym środowisku condy
# import sys
# !conda install -c conda-forge --yes --prefix {sys.prefix} spacy

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\pawel\anaconda3\envs\hahaha

  added / updated specs:
    - spacy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aiohttp-3.7.4              |   py38h294d835_0         596 KB  conda-forge
    boto-2.49.0                |             py_0         838 KB  conda-forge
    boto3-1.17.57              |     pyhd8ed1ab_0          70 KB  conda-forge
    botocore-1.20.57           |     pyhd8ed1ab_0         4.6 MB  conda-forge
    brotlipy-0.7.0             |py38h294d835_1001         368 KB  conda-forge
    catalogue-2.0.3            |   py38haa244fe_0          31 KB  conda-forge
    chardet-4.0.0              |   py38haa244fe_1         224 KB  conda-forge
    cryptography-3.4.7         |   py38hd7da0ea_0         706 KB  conda-forge
    cymem-2.0.5                |   py38h885f38d_1          40 KB  cond

In [26]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish

nlp = Polish()

In [27]:
nlp

In [28]:
tokenizer = Tokenizer(nlp.vocab)

In [96]:
PATH = "romeo-i-julia-700.txt"
with open(PATH, "r", encoding="UTF-8") as file:
    text = file.readlines()

## Zad 6

In [103]:
import random

def remove_random_tokens(text, path, percent):
    pipe = tokenizer.pipe(text)
    with open(path, "w", encoding="UTF-8") as file:
        for doc in pipe:
            for token in doc:
                if random.random() >= percent/100:
                    file.write(token.text_with_ws)

remove_random_tokens(text, "romeo_trimmed_1.txt", 3)
remove_random_tokens(text, "romeo_trimmed_2.txt", 3)

## Zad 7

In [104]:
def get_list_of_tokens(text):
    pipe = tokenizer.pipe(text)
    result = []
    for doc in pipe:
        for token in doc:
            result.append(token.text)
    return result

with open("romeo_trimmed_1.txt", "r", encoding="UTF-8") as file:
    list_of_tokens_1 = get_list_of_tokens(file.readlines())
with open("romeo_trimmed_2.txt", "r", encoding="UTF-8") as file:
    list_of_tokens_2 = get_list_of_tokens(file.readlines())



lcs_length, _ = lcsequence(list_of_tokens_1, list_of_tokens_2)
print(f'Liczba tokenów 1. tekstu: {len(list_of_tokens_1)}')
print(f'Liczba tokenów 2. tekstu: {len(list_of_tokens_2)}')
print(f'Długość najdłuższego wspólnego podciągu: {lcs_length}')

Liczba tokenów 1. tekstu: 2568
Liczba tokenów 2. tekstu: 2545
Długość najdłuższego wspólnego podciągu: 2485


## Zad 8

In [107]:
def diff(seq_a, seq_b):
    _, traceback = lcsequence(seq_a, seq_b)
    line_a, line_b = len(seq_a), len(seq_b)

    differences = []
    while line_a > 0 and line_b > 0:
        if traceback[line_a][line_b] == "diag":
            line_a, line_b = line_a - 1, line_b - 1
        elif traceback[line_a][line_b] == "up":
            differences.append(f'< [{line_a}] {seq_a[line_a-1]} ')
            line_a -= 1
        else:
            differences.append(f'> [{line_b}] {seq_b[line_b-1]}')
            line_b -= 1

    while line_a > 0:
        differences.append(f'< [{line_a}] {seq_a[line_a]}')
        line_a -= 1
    while line_b > 0:
        differences.append(f'> [{line_b}] {seq_b[line_b]}')
        line_b -= 1

    differences.reverse()
    return differences

## Zad 9

In [109]:
def get_lines(path):
    with open(path, "r", encoding="UTF-8") as file:
        return [line.strip() for line in file.readlines()]

lines_text_1 = get_lines("romeo_trimmed_1.txt")
lines_text_2 = get_lines("romeo_trimmed_2.txt")


result = diff(lines_text_1, lines_text_2)
for line in result:
    print(line)

> [12] * PARYS — młody Weroneńczyk szlachetnego rodu, krewny księcia
< [12] * PARYS — młody Weroneńczyk szlachetnego rodu, krewny 
> [14] * STARZEC — brat Kapuleta
> [15] * ROMEO — syn Montekiego * MERKUCJO — krewny księcia
< [14] * STARZEC — stryjeczny brat Kapuleta 
< [15] * ROMEO — syn Montekiego 
< [16] * MERKUCJO — krewny księcia 
> [18] * LAURENTY — ojciec
> [19] * JAN — brat z tegoż zgromadzenia
< [19] * LAURENTY — ojciec franciszkanin 
< [20] * — brat z tegoż zgromadzenia 
> [28] * PANI MONTEKI — małżonka
< [29] * PANI MONTEKI — małżonka Montekiego 
> [50] Z łon tych dwu wzięło bowiem życie,
< [51] Z łon tych dwu wrogów wzięło bowiem życie, 
> [60] Które otoczcie cierpliwymi względy,
> [61] Jest w nim co złego, my usuniem błędy…
< [61] Które otoczcie względy, 
< [62] Jest w nim złego, my usuniem błędy… 
> [72] / Plac publiczny. Wchodzą Samson i Grzegorz uzbrojeni tarcze i miecze. /
< [73] / Plac publiczny. Wchodzą Samson i Grzegorz uzbrojeni w tarcze i miecze. / 
> [76] 
> [93]